In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

train_dataset = pd.read_csv('../input/30-days-of-ml/train.csv', index_col=0)
test_dataset = pd.read_csv('../input/30-days-of-ml/test.csv', index_col=0)

# Define and seperate target from the train_dataset
y = train_dataset.target
X = train_dataset.drop('target', axis=1)

# Deploy train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [3]:
# define categorical columns
cat_col = [cname for cname in X_train.columns if X_train[cname].nunique() <= 10 and X_train[cname].dtype == 'object']

# define numerical columns
num_col = [cname for cname in X_train.columns if X_train[cname].dtype in ['Int64', 'float64']]

#selected_cols = cat_col + num_col
#X_train = X_train[selected_cols]
#X_valid = X_valid[selected_cols]
#X_test = test_dataset[selected_cols]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """


# Deploying OneHot Encoding Technique for data preparation

In [ ]:
""" from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

Oh_col_train = pd.DataFrame(OH_encoder.fit_transform(X_train[cat_col]))
Oh_col_valid = pd.DataFrame(OH_encoder.transform(X_valid[cat_col]))

# OneHot encoding removes index, so I put it back
Oh_col_train.index = X_train.index
Oh_col_valid.index = X_valid.index

# Remove categorical columns
num_X_train = X_train.drop(cat_col, axis=1)
num_X_valid = X_valid.drop(cat_col, axis=1)

# Add OneHot encoding to numerical features
OH_X_train = pd.concat([num_X_train, Oh_col_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, Oh_col_valid], axis=1)

# Building Pipeline and Deploying Cross Validation

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_col),
        ('cat', categorical_transformer, cat_col)
    ])

In [5]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error


#random seed
rans = 42

In [6]:
xgb_params = {'n_estimators': 10000, 'max_depth': 3, 'learning_rate': 0.03628302216953097, 'gamma': 0,
              'min_child_weight': 1, 'subsample': 0.7875490025178415, 'colsample_bytree': 0.11807135201147481,
              'reg_alpha': 23.13181079976304, 'reg_lambda': 0.0008746338866473539, 'random_state':rans,
              'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor'}

gxgb_paramsd = {'n_estimators': 4575, 'max_depth': 3, 'learning_rate': 0.03928410410450762,
              'gamma': 0.30000000000000004, 'min_child_weight': 3, 'subsample': 0.8, 'colsample_bytree': 0.6,
              'reg_alpha': 0.2, 'reg_lambda': 0.5, 'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor'}

vxgb_params = {'n_estimators': 4693, 'max_depth': 8, 'learning_rate': 0.007701251349251828,
              'gamma': 0.7000000000000001, 'min_child_weight': 1, 'subsample': 0.7, 'colsample_bytree': 0.6,
              'reg_alpha': 0.6000000000000001, 'reg_lambda': 0.7000000000000001,'random_state':rans,
             'tree_method': 'gpu_hist', 'gpu_id': 0, 'predictor': 'gpu_predictor'}

model = XGBRegressor(**xgb_params)
#tree_method='gpu_hist', gpu_id=0, predictor="gpu_predictor"

In [7]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [8]:
my_pipeline.fit(X_train, y_train)

pred = my_pipeline.predict(X_valid)

score = mean_squared_error(y_valid, pred, squared=False)

In [9]:
score

0.7197755735042503

In [10]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_squared_error')

In [12]:
#score = mean_squared_error(y_valid, pre, squared=False)

In [13]:
scores.mean()

0.5152483290962826

In [ ]:
score